In [ ]:
# this should be the set to false for production use
# and set to true for testing (default) but can be already set to false from outside (e.g., by the test script)

if "doRunTests" not in globals():
    doRunTests = True
    pathToData = "dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1"


In [ ]:
# rename files

import os
import glob


def change_TaskToken(fullFileName, visit_token, new_visit_token):
    """change the visit token in the file name"""
    fname = os.path.splitext(os.path.basename(fullFileName))[0]
    fext = os.path.splitext(os.path.basename(fullFileName))[1]
    fpath = os.path.dirname(fullFileName)
    new_fname = fname.replace(visit_token, new_visit_token)
    if fname != new_fname:
        rename_file(fname, new_fname, fext, fpath)
    full_fname = os.path.join(fpath, fname + fext)
    full_new_fname = os.path.join(fpath, new_fname + fext)
    return full_fname, full_new_fname


def print_output(fname1, fname2, pathToVisit):
    """print the original and new file names"""
    relativePath1 = os.path.relpath(fname1, pathToVisit)
    relativePath2 = os.path.relpath(fname2, pathToVisit)
    relativePath2 = os.path.normpath(relativePath2)
    if fname1 != fname2:
        print("    " + relativePath1)
        print(" -> " + relativePath2)
    # else:
    #     print("    " + fname1 + " -> unchanged")
    # print(" ")


def fname_without_double_extension(fname, fext):
    """return the file name without double extension"""
    if fname.endswith(fext):
        return fname.replace(fext, "")
    return fname


def rename_file(fname, new_fname, fext, absolutePath):
    """rename the file fname to new_fname with extension fext in the absolutePath"""
    full_fname = os.path.join(absolutePath, fname + fext)
    full_new_fname = os.path.join(absolutePath, new_fname + fext)
    # os.rename(full_fname, full_new_fname)


def remove_double_extension(fullFileName):
    """remove a possible double extension in fileName"""
    fname = os.path.splitext(os.path.basename(fullFileName))[0]
    fext = os.path.splitext(os.path.basename(fullFileName))[1]
    fpath = os.path.dirname(fullFileName)
    new_fname = fname_without_double_extension(fname, fext)
    if fname != new_fname:
        rename_file(fname, new_fname, fext, fpath)

    full_fname = os.path.join(fpath, fname + fext)
    full_new_fname = os.path.join(fpath, new_fname + fext)
    return full_fname, full_new_fname


def reorder_tokens_in_file_name(fullFileName, task_token):
    """reorder the tokens in the file name"""
    fname = os.path.splitext(os.path.basename(fullFileName))[0]
    fext = os.path.splitext(os.path.basename(fullFileName))[1]
    fpath = os.path.dirname(fullFileName)
    tokens = fname.split("_")
    new_fname = fname
    # change the token order to Rearm_Patient_Date_Visit_Task_...
    # if the last token is the task token
    if tokens[-1] == task_token and tokens[4] != task_token:
        # insert the last token after the visit token (4th token)
        new_fname = (
            tokens[0]
            + "_"
            + tokens[1]
            + "_"
            + tokens[2]
            + "_"
            + tokens[3]
            + "_"
            + tokens[-1]
        )
        # if there are more than 5 tokens,
        # add the extra tokens to the new file name (omit the last token)
        if len(tokens) > 5:
            for i in range(4, len(tokens) - 1):
                new_fname = new_fname + "_" + tokens[i]

    # if fname and new_fname are different, rename the file
    if fname != new_fname:
        rename_file(fname, new_fname, fext, fpath)
    full_fname = os.path.join(fpath, fname + fext)
    full_new_fname = os.path.join(fpath, new_fname + fext)
    return full_fname, full_new_fname


if doRunTests:
    relativePath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/ReArm_C1P02_20210306_V1_Circle"
    task_token = "c"
    
    relativePath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/ReArm_C1P02_20210306_V1_Reaching"
    task_token = "r"

    absolutePath = os.path.abspath(relativePath)

    print("Reorder tokens in file name")
    absFilePathList = glob.glob(absolutePath + "/*")
    for absFileName in absFilePathList:
        fullFile = os.path.join(absolutePath, os.path.basename(absFileName))
        f_in_r, f_out_r = reorder_tokens_in_file_name(fullFile, task_token)
        print_output(f_in_r, f_out_r, absolutePath)

    print("Remove double extension in file names")
    absFilePathList = glob.glob(absolutePath + "/*")
    for absFileName in absFilePathList:
        fullFile = os.path.join(absolutePath, os.path.basename(absFileName))
        f_in, f_new = remove_double_extension(fullFile)
        print_output(f_in, f_new, absolutePath)


In [ ]:
def list_all_absFilePath(absVisitPath):
    """
    Sets the list of absolute path to all files in the visit directory.
    """
    allFiles = glob.glob(os.path.join(absVisitPath, "**/*"), recursive=True)
    allFiles = [f for f in allFiles if not os.path.isdir(f)]
    allFiles = [os.path.abspath(f) for f in allFiles]
    return allFiles


def remove_double_extension_in_all_files(absVisitPath):
    """
    Remove double extension in all file names in the visit directory.
    """
    absFilePathList = list_all_absFilePath(absVisitPath)
    for absFileName in absFilePathList:
        f_in, f_new = remove_double_extension(absFileName)
        print_output(f_in, f_new, absVisitPath)


def rename_task_token_in_all_files(absVisitPath):
    """
    Rename task token in all file names in the visit directory.
    """
    absFilePathList = list_all_absFilePath(absVisitPath)
    for absFileName in absFilePathList:
        if "_Accelerometry" in os.path.dirname(absFileName):
            f_in_a, f_out_a = change_TaskToken(absFileName, "Accelerometry", "ac")
            print_output(f_in_a, f_out_a, absVisitPath)


def reorder_tokens_in_all_files(absVisitPath):
    """
    Reorder tokens in all file names in the visit directory.
    """
    absFilePathList = list_all_absFilePath(absVisitPath)
    for absFileName in absFilePathList:
        full_fpath = os.path.dirname(absFileName)
        f_in_r = absFileName
        f_out_r = absFileName
        if "_Reaching" in full_fpath:
            f_in_r, f_out_r = reorder_tokens_in_file_name(absFileName, "r")
        if "_Circle" in full_fpath:
            f_in_r, f_out_r = reorder_tokens_in_file_name(absFileName, "c")
        if "_Accelerometry" in full_fpath:
            f_in_r, f_out_r = reorder_tokens_in_file_name(absFileName, "ac")
        if "_Armeo" in full_fpath:
            f_in_r, f_out_r = reorder_tokens_in_file_name(absFileName, "a")

        print_output(f_in_r, f_out_r, absVisitPath)

def renameFilesInVisit(visitPath):
    absVisitPath = os.path.abspath(visitPath)

    print("Remove double extension in all file names in the visit directory")
    remove_double_extension_in_all_files(absVisitPath)

    print("Rename task token in all file names in the visit directory")
    rename_task_token_in_all_files(absVisitPath)

    print("Reorder tokens in all file names in the visit directory")
    reorder_tokens_in_all_files(absVisitPath) 

if doRunTests:
    visitPath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/"
    # visitPath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210419_V2/"
    # visitPath = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210715_V3/"

    absVisitPath = os.path.abspath(visitPath)

    print("Remove double extension in all file names in the visit directory")
    remove_double_extension_in_all_files(absVisitPath)

    print("Rename task token in all file names in the visit directory")
    rename_task_token_in_all_files(absVisitPath)

    print("Reorder tokens in all file names in the visit directory")
    reorder_tokens_in_all_files(absVisitPath)